In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print('Dependencies Imported')

Dependencies Imported


In [2]:
data = pd.read_csv("Coursera.csv")
data=data[['Course Name','Difficulty Level','Course Rating','Course Description','Skills']]
data.head(5)

,Course Name,Difficulty Level,Course Rating,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,4.8,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,4.8,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,4.1,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,4.8,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,4.6,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [3]:
# Convert course_rating columns to float
data['Course Rating'] = pd.to_numeric(data['Course Rating'], errors='coerce')

In [4]:
data = data[['Course Name','Difficulty Level','Course Description','Skills']]
data.head(5)

,Course Name,Difficulty Level,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Combine course name and description into a single text column for vectorization
data['combined'] = data['Course Name'] + ' ' + data['Course Description'] + ' ' + data['Skills']

# Initialize the TF-IDF Vectorizer to convert text data into numerical format
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the combined text data to create a TF-IDF matrix
X = tfidf_vectorizer.fit_transform(data['combined'])

# Initialize the Nearest Neighbors model
model = NearestNeighbors(n_neighbors=4, metric='euclidean') 

model.fit(X)

def recommend_courses(course_name):
    """
    Recommend courses based on the input course name using Nearest Neighbors.

    Parameters:
    course_name (str): The name of the course for which recommendations are to be made.

    Returns:
    list: A list of recommended course names based on similarity.
    """
    # Transform the input course name into the TF-IDF format
    course_vector = tfidf_vectorizer.transform([course_name])

    # Find the nearest neighbors for the input course vector
    distances, indices = model.kneighbors(course_vector)

    # Extract recommended course names based on the indices of nearest neighbors
    recommended_courses = []
    for index in indices[0]:
        recommended_courses.append(data.iloc[index]['Course Name'])

    return recommended_courses

# Example usage
if __name__ == "__main__":
    # Define a course name to get recommendations for
    course_to_predict = 'HTML & CSS'

    # Get recommended courses based on the specified course
    recommendations = recommend_courses(course_to_predict)

    # Print the recommended courses
    print("Recommended Courses:", recommendations)


Recommended Courses: ['HTML and CSS: Building a Single-Page Website', 'Responsive Website Basics: Code with HTML, CSS, and JavaScript', 'Build Your Portfolio Website with HTML and CSS', 'Getting Started with Cascading Style Sheet']


In [8]:
def evaluate_recommendations(test_courses, recommended_courses):
    """
    Evaluate the recommendations using precision and recall.

    Parameters:
    test_courses (list): The actual relevant courses that the user liked.
    recommended_courses (list): The courses recommended by the model.

    Returns:
    tuple: Precision and Recall scores.
    """
    # Convert the lists to sets for easy comparison
    true_set = set(test_courses)
    recommended_set = set(recommended_courses)

    # Calculate True Positives: Relevant courses that were recommended
    true_positives = len(true_set & recommended_set)

    # Precision: The ratio of true positives to the total number of recommended courses
    precision = true_positives / len(recommended_set) if recommended_set else 0

    return precision

# Example usage
actual_courses = [
    'Python Basics', 
    'Python Data Analysis', 
    'Python Programming Essentials',
    'Python Data Representations'
]

# Get recommendations for a course (e.g., 'HTML & CSS')
predicted_courses = recommend_courses('Python')

# Print recommended courses
print("Recommended Courses:", predicted_courses)

# Evaluate the recommendations
precision = evaluate_recommendations(actual_courses, predicted_courses)

# Print the precision and recall scores
print(f"Precision: {precision:.2f}")

Recommended Courses: ['Python Data Analysis', 'Python Data Representations', 'Python Programming Essentials', 'Python Data Structures']
Precision: 0.75


In [10]:
import mlflow
mlflow.set_experiment("Learning Path Recommendation")
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

with mlflow.start_run(run_name="KNN Algorithm"):
    mlflow.log_params({
        "n_neighbors": 4, 
        "metric": 'euclidean'
    })
    mlflow.log_metric('Precision', precision) # type: ignore
    mlflow.sklearn.log_model(model, "KNN recommendation model")

2024/10/28 12:04:00 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/10/28 12:04:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/28 12:04:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run KNN Algorithm at: http://127.0.0.1:5000/#/experiments/725882505928502647/runs/5f3b8b433d0444e7a2e1d8a92fb5d043.
2024/10/28 12:04:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/725882505928502647.


In [11]:
model_uri = 'runs:/5f3b8b433d0444e7a2e1d8a92fb5d043/KNN recommendation model'
mlflow.register_model(model_uri,"KNN recommendation model")

Successfully registered model 'KNN recommendation model'.
2024/10/28 12:05:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: KNN recommendation model, version 1
Created version '1' of model 'KNN recommendation model'.


<ModelVersion: aliases=[], creation_timestamp=1730097358072, current_stage='None', description='', last_updated_timestamp=1730097358072, name='KNN recommendation model', run_id='5f3b8b433d0444e7a2e1d8a92fb5d043', run_link='', source=('file:///c:/Users/AkhilKumar/Tasks/FinalProject%20DE/DataScience/mlruns/725882505928502647/5f3b8b433d0444e7a2e1d8a92fb5d043/artifacts/KNN '
 'recommendation model'), status='READY', status_message='', tags={}, user_id='', version='1'>

In [12]:
model_name="KNN recommendation model"
dev_model_uri=f'models:/{model_name}@challenger'
prod_model='KNN recommendation model_Production'
  
client=mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri,dst_name=prod_model)

Successfully registered model 'KNN recommendation model_Production'.
Copied version '1' of model 'KNN recommendation model' to version '1' of model 'KNN recommendation model_Production'.


<ModelVersion: aliases=[], creation_timestamp=1730097404009, current_stage='None', description='', last_updated_timestamp=1730097404009, name='KNN recommendation model_Production', run_id='5f3b8b433d0444e7a2e1d8a92fb5d043', run_link='', source='models:/KNN recommendation model/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [17]:
m2 = mlflow.sklearn.load_model(f'models:/{prod_model}@champion')
recommendations = recommend_courses("html")
recommendations

['HTML and CSS: Building a Single-Page Website',
 'Quick resumeCreator with JavaScript',
 'Responsive Website Basics: Code with HTML, CSS, and JavaScript',
 'Build Your Portfolio Website with HTML and CSS']